In [13]:
import pandas as pd
pd.set_option('display.max_columns', 50)

In [20]:
data = pd.read_csv('heart_num_0228.csv')
print(len(data))
data.head()

274034


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,HighRiskLastYear,CovidPos,"RaceEthnicityCategory_Black only, Non-Hispanic",RaceEthnicityCategory_Hispanic,"RaceEthnicityCategory_Multiracial, Non-Hispanic","RaceEthnicityCategory_Other race only, Non-Hispanic","RaceEthnicityCategory_White only, Non-Hispanic","TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap"
0,0,0,4.0,0.0,0.0,3.0,0,6.0,3.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0,0,80,1.60,68.04,26.57,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,0,0,1.0,2.0,0.0,3.0,1,9.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0,1,40,1.57,53.98,21.77,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0
2,0,1,0.0,1.0,0.0,3.0,0,7.0,3.0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,3.0,3.0,0,80,1.80,84.82,26.08,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
3,0,0,3.0,0.0,0.0,3.0,1,7.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3.0,0,80,1.65,62.60,22.96,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,0,0,2.0,0.0,0.0,3.0,0,8.0,3.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3.0,3.0,1,80,1.63,73.48,27.81,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0


In [21]:
data.isna().sum()

State                                                                          0
Sex                                                                            0
GeneralHealth                                                                  0
PhysicalHealthDays                                                             0
MentalHealthDays                                                               0
LastCheckupTime                                                                0
PhysicalActivities                                                             0
SleepHours                                                                     0
RemovedTeeth                                                                   0
HadHeartAttack                                                                 0
HadAngina                                                                      0
HadStroke                                                                      0
HadAsthma                   

In [22]:
Q1 = data[['SleepHours', 'BMI']].quantile(q = 0.25)
Q3 = data[['SleepHours', 'BMI']].quantile(q = 0.75)
print(Q1)
print()
print(Q3)

SleepHours     6.00
BMI           24.21
Name: 0.25, dtype: float64

SleepHours     8.00
BMI           31.89
Name: 0.75, dtype: float64


In [23]:
IQR = Q3-Q1

In [24]:
data = data[(data['SleepHours'] <= Q3['SleepHours']+IQR['SleepHours']*1.5)&(data['SleepHours'] >= Q1['SleepHours']-IQR['SleepHours']*1.5)]
data = data[(data['BMI'] <= Q3['BMI']+IQR['BMI']*1.5)&(data['BMI'] >= Q1['BMI']-IQR['BMI']*1.5)]
print(len(data))

261812


In [26]:
from sklearn.preprocessing import Normalizer

num_col = ['PhysicalHealthDays', 'MentalHealthDays', 'SleepHours', 'HeightInMeters', 'WeightInKilograms', 'BMI']
df_num = data[num_col]
norm = Normalizer()
norm.fit(df_num)
df_normScaled = norm.transform(df_num)
df_norm = pd.DataFrame(df_normScaled, columns = ['PhysicalHealthDays', 'MentalHealthDays', 'SleepHours', 'HeightInMeters', 'WeightInKilograms', 'BMI'])
df_norm.head()

,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI
0,0.000000,0.0,0.081847,0.021826,0.928147,0.362447
1,0.033926,0.0,0.152669,0.026632,0.915673,0.369289
2,0.011231,0.0,0.078618,0.020216,0.952623,0.292907
3,0.000000,0.0,0.104377,0.024603,0.933430,0.342357
4,0.000000,0.0,0.101279,0.020636,0.930249,0.352071


In [28]:
data_cat = data.drop(num_col, axis = 1)
data = pd.concat([df_norm, data_cat], axis = 1)
print(len(data.columns))
data.head()

47


,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI,State,Sex,GeneralHealth,LastCheckupTime,PhysicalActivities,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,AgeCategory,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,HighRiskLastYear,CovidPos,"RaceEthnicityCategory_Black only, Non-Hispanic",RaceEthnicityCategory_Hispanic,"RaceEthnicityCategory_Multiracial, Non-Hispanic","RaceEthnicityCategory_Other race only, Non-Hispanic","RaceEthnicityCategory_White only, Non-Hispanic","TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap"
0,0.000000,0.0,0.081847,0.021826,0.928147,0.362447,0.0,0.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.033926,0.0,0.152669,0.026632,0.915673,0.369289,0.0,0.0,1.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,40.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.011231,0.0,0.078618,0.020216,0.952623,0.292907,0.0,1.0,0.0,3.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,80.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.000000,0.0,0.104377,0.024603,0.933430,0.342357,0.0,0.0,3.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,80.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.000000,0.0,0.101279,0.020636,0.930249,0.352071,0.0,0.0,2.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,80.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [31]:
print(len(data_cat))
print(len(df_norm))

261812
261812


In [10]:
X = data.drop('HadHeartAttack', axis = 1)
y = data['HadHeartAttack']

In [18]:
print(data.shape)
print(X.shape)
print(y.shape)

(273456, 47)
(273456, 46)
(273456,)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [12]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter = 10000)
sgd.fit(X_train, y_train)

ValueError: Input y contains NaN.